# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [4]:
%matplotlib inline

In [2]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from dateutil.parser import parse

In [3]:
DATA_FOLDER = './data/'

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

- By studying the data we decided to only choose the confirmed cases and deaths. The reason herefor is that there is no way to know, if a patient who is previously registered as a suspected case, gets counted again when his illness is confirmed to be ebola. Analogous for the deaths, maybe one day a death gets registered as potentially ebola and days later after more thorough medical analysis it is confirmed to have been ebola, so another death gets counted in the confirmed cases. We did not want this to occur.



1. Create new Data Frame with wanted attributes
2. Provide functions that reads all csvs from a given directory
3. Append wanted attributes to new data frame (only confirmed)
4. Group data by Month and country and calculate the mean on the other fields. 

In [28]:
ebola = pd.DataFrame(columns=['Month', 'Country', 'New', 'Deaths'])

def ebola_dirs(dirname, country, desc_col, totals_col, new_id, deaths_id):
    global ebola
    
    path = DATA_FOLDER + 'ebola/' + dirname
    old_sierra = 0
    for csv in sorted(os.listdir(path)):
        frame = pd.read_csv(path + '/' + csv).fillna(0)
        frame = frame.replace(to_replace='.*%$', value='0', regex=True)
        frame = frame.replace(to_replace='.*\..*$', value='0', regex=True)
        frame = frame.replace(to_replace='.*\,.*', value='0', regex=True)
        
        if country == 'Liberia' and '2014-12' in csv:
            continue
        
        deaths_id_mod = deaths_id
        if country == 'Guinea' and csv == '2014-08-04.csv':
            deaths_id_mod += ' today'
        
        month = parse(frame[frame.columns[0]][0]).month
        new = int(frame[frame[desc_col] == new_id][totals_col])
        deaths = 0
        if country == 'Sierra Leone':
            deaths1 = int(frame[frame[desc_col] == deaths_id_mod][totals_col])
            print(csv, deaths1, old_sierra)
            if deaths1 != 0:
                deaths = deaths1 - old_sierra
                old_sierra = deaths1
            else:
                deaths = 0
        else:
            deaths = int(frame[frame[desc_col] == deaths_id_mod][totals_col])
        
        row = pd.DataFrame({'Month': month, 'Country': country, 'New': new, 'Deaths': deaths}, index=[0])
        ebola = ebola.append(row, ignore_index=True)

ebola_dirs('guinea_data/', 'Guinea', 'Description', 'Totals', 'New cases of confirmed', 'New deaths registered')
ebola_dirs('liberia_data/', 'Liberia', 'Variable', 'National', 'New case/s (confirmed)', 'Newly reported deaths')
ebola_dirs('sl_data/', 'Sierra Leone', 'variable', 'National', 'new_confirmed', 'death_confirmed')

ebola["Deaths"] = pd.to_numeric(ebola["Deaths"])
ebola["New"] = pd.to_numeric(ebola["New"])
ebola.groupby(['Month', 'Country']).agg({'New': np.mean, 'Deaths': np.mean})
#ebola

2014-08-12-v77.csv 264 0
2014-08-13-v78.csv 273 264
2014-08-14-v79.csv 280 273
2014-08-15-v80.csv 287 280
2014-08-16-v81.csv 297 287
2014-08-17-v82.csv 305 297
2014-08-18-v83.csv 312 305
2014-08-19-v84.csv 320 312
2014-08-20-v85.csv 322 320
2014-08-21-v86.csv 329 322
2014-08-22-v87.csv 333 329
2014-08-23-v88.csv 336 333
2014-08-24-v89.csv 341 336
2014-08-25-v90.csv 355 341
2014-08-26-v91.csv 0 355
2014-08-27-v92.csv 372 355
2014-08-28-v93.csv 377 372
2014-08-29-v94.csv 0 377
2014-08-30-v95.csv 387 377
2014-08-31-v96.csv 387 387
2014-09-01-v97.csv 396 387
2014-09-02-v98.csv 399 396
2014-09-03-v99.csv 404 399
2014-09-05-v101.csv 0 404
2014-09-06-v102.csv 426 404
2014-09-07-v103.csv 428 426
2014-09-08-v104.csv 433 428
2014-09-09-v105.csv 436 433
2014-09-10-v106.csv 445 436
2014-09-11-v107.csv 450 445
2014-09-12-v108.csv 0 450
2014-09-13-v109.csv 463 450
2014-09-14-v110.csv 468 463
2014-09-15-v111.csv 474 468
2014-09-16-v112.csv 483 474
2014-09-17-v113.csv 489 483
2014-09-18-v114.csv 495 4

New     Deaths
Month Country                           
6     Liberia        2.142857   2.000000
7     Liberia        1.818182   4.272727
8     Guinea        12.400000   3.400000
      Liberia        5.444444  23.222222
      Sierra Leone  18.550000  19.350000
9     Guinea        13.000000   3.562500
      Liberia        6.166667  36.041667
      Sierra Leone  34.413793   5.620690
10    Guinea         6.000000  15.000000
      Liberia        1.360000  28.040000
      Sierra Leone  56.714286  18.571429
11    Liberia        2.600000  13.466667
      Sierra Leone  59.904762  20.285714
12    Sierra Leone  32.600000  42.400000

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

For each excel file, we:
    - Read the excel file into a pandas data frame
    - Name the columns and append a new Barcode column to it, where every entry is the name of the current file
    - Append this data frame to the rna data frame collecting the data of all 10 files
    
Then read the meta excel file into a data frame and do an inner join with the rna data frame on the "BARCODE" column. We replaced all the NaN values with "unknown" and create a unique index.

In [ ]:
# Task 2

rna = pd.DataFrame()

for i in range(1, 10):
    file = 'MID' + str(i)
    frame = pd.read_excel(DATA_FOLDER + '/microbiome/' + file + '.xls')
    frame.columns = ['TYPE', 'NUMBER']
    frame['BARCODE'] = file
    frame.index = list(map(lambda n: file + '_' + str(n), np.arange(1, len(frame)+1)))
    rna = pd.concat([rna, frame])
    
meta = pd.read_excel(DATA_FOLDER + '/microbiome/metadata.xls', index_col=0)

rna.join(meta, on="BARCODE").drop('BARCODE', axis=1).fillna('unknown')

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [12]:
titanic = pd.read_excel(DATA_FOLDER + '/titanic.xls')

### 3.1

| Attribute     | Type     | Range         | Categorical     |
|---------------|----------|---------------|-----------------|
|pclass         |int64     |[1, 3]         |Yes              |
|survived       |int64     |[0, 1]         |Yes              |
|name           |object    |-              |No               |
|sex            |object    |{male, female} |Yes              |
|age            |float64   |[0.1667, 80.0] |No               |
|sibsp          |int64     |[0, 8]         |No               |
|parch          |int64     |[0, 9]         |No               |
|ticket         |object    |-              |Yes              |
|fare           |float64   |[0.0, 512.33]  |No               |
|cabin          |object    |-              |Yes              |
|embarked       |object    |{S, C, Q}      |Yes              |
|boat           |object    |-              |Yes              |
|body           |float64   |[1.0, 328.0]   |No               |
|home.dest      |object    |-              |No               |

In [13]:
titanic.pclass = titanic.pclass.astype('category')
titanic.survived = titanic.survived.astype('category')
titanic.ticket = titanic.ticket.astype('category')
titanic.cabin = titanic.cabin.astype('category')
titanic.sex = titanic.sex.astype('category')
titanic.boat = titanic.boat.astype('category')
titanic.embarked = titanic.embarked.astype('category')

In [ ]:
# 2. Miscellaneous histograms

plt.figure(figsize=(16, 4))

# Travel class
plt.subplot(141)
tc = titanic.pclass.value_counts().plot(kind='bar')

# Embarkation point
titanic.embarked.replace(['S', 'C', 'Q'], ['Southampton', 'Cherbourg', 'Queenstown'], inplace=True)
plt.subplot(142)
em = titanic.embarked.value_counts().plot(kind='bar')

# Sex 
plt.subplot(143)
sx = titanic.sex.value_counts().plot(kind='bar')

# Age
plt.subplot(144)
ae = titanic.age.hist(bins=np.arange(0, 100, 10))

In [ ]:
# 3. Cabin floor allocation

titanic.cabin.dropna().map(lambda cf: str(cf)[0]).value_counts().plot.pie(autopct='%.1f%%', pctdistance=1.1, labeldistance=1.3)

In [ ]:
#titanic[['pclass', 'survived']].stack().astype(int).unstack()

In [ ]:
# 4. Survivors per cabin floor

sv = titanic[['pclass', 'survived']].stack().astype(int).unstack().groupby(['pclass']).mean()

plt.figure(figsize=(16, 4))

plt.subplot(131)
plt.pie([sv.loc[1], 1 - sv.loc[1]], labels=['a', 'b'], autopct='%.1f%%', pctdistance=1.1, labeldistance=1.3)

plt.subplot(132)
plt.pie([sv.loc[2], 1 - sv.loc[2]])

plt.subplot(133)
plt.pie([sv.loc[3], 1 - sv.loc[3]])

In [ ]:
sv = titanic[['survived', 'pclass', 'sex']].groupby(['pclass', 'sex']).sum()
st = titanic[['pclass', 'sex']].groupby(['pclass', 'sex']).size()
sv['died'] = st-sv.survived
sv.plot.bar(stacked=True)

In [16]:
cat = titanic.sort_values('age')[['age', 'survived', 'sex', 'pclass']].dropna()
cat['cat'] = pd.qcut(cat['age'].values, 2, labels=['young', 'old'])
cat = cat[['cat', 'survived', 'pclass', 'sex']]
cat["survived"] = pd.to_numeric(cat["survived"])
cat.groupby(['pclass', 'sex', 'cat']).mean()

survived
pclass sex    cat            
1      female old    0.967391
              young  0.951220
       male   old    0.302521
              young  0.531250
2      female old    0.872340
              young  0.910714
       male   old    0.083333
              young  0.216216
3      female old    0.372093
              young  0.513761
       male   old    0.144000
              young  0.183036